I'll be cleaning the data appropiately and saving the subdata as seperate csv files. This is required to complete the surivial analysis and the additional calculations.

I performed the data cleaning with python rather than R since I was more familiar with it at the time of writing the code.

Import necessary libraries 

In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re 

print("complete")

complete


Read csv file 

In [2]:
df = pd.read_csv("/Users/anthonyquint/Desktop/LHSC_Work_Folder/Mina/MM_ACST_Study/ASCT_Study_July 24, MinaShona Edit.csv")
#df.head()

Other data cleaning stuff

In [3]:
#Only maintain columns and rows I want 

#Choosing only the important columns 
df = df[['STUDY_PATIENT_ID','Age','GENDER','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','Overall Survival (OS) (days)', 'OS_Status (1 = censored, 2 = event)', 'Progression Free Survival 2 (PFS2) (days)', 'PFS2_Status (1 = censored, 2 = event)', 'At_Risk (Y/N) for Anthony','At risk Y/N', 'Hemoglobin', 'Platelets','Neutrophils', 'Creatinine', 'Calcium','TANDEM','REGIMEN_NAME_maintenance','Len. Maint. Dose (Dummary var for 1)','Len. Maint. Dose (Dummary var for 2)', 'lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)','Num. Maint. Drugs. (Dummy var for 1)','Num. Maint. Drugs. (Dummy var for 2)', 'Number_of_Maintenance_Drugs','Response post induction', 'ASCT response  1-3 months', 'ASCT response 12 months', 'Response post 2nd induction', 'ASCT response  1-3 months (second transplant)', 'ASCT response 12 months (second transplant)', 'Tandem 1 response 1-3 months', 'Tandem 2 response 1-3 months', 'Tandem 2 response 12 months', 'Maint. Response 12 months', 'Maint. Response 12 months (2nd transplant)', 'Maint. Type (after Tandem)', 'Paatient Status', 'Comments']]

#Exclusion criteria: eliminating rows of people who had thalimide maintenance 
df = df[~df['REGIMEN_NAME_maintenance'].isin(['Thalidomide'])]
df = df[~df['REGIMEN_NAME_maintenance'].isin(['TD'])]
df = df[~df['REGIMEN_NAME_maintenance'].isin(['TP'])]
print(df['REGIMEN_NAME_maintenance'].value_counts(dropna=False))


NaN             105
Revlimid         24
Velcade           9
RV                7
Lenalidomide      6
IxaR              3
RD                1
ixazomib          1
IxaRD             1
RVD               1
Name: REGIMEN_NAME_maintenance, dtype: int64


Saving master data file

In [4]:
#df.to_csv("ASCT_Master_Dataset.csv") # Normally not commented out

Creating the sub data files that will be used for the specific analysis - Kapler Meier Curves 

Normally the ".to_csv" lines of code aren't commented out.

In [5]:
### PFS, high risk patients, (Li + Pi vs. single vs. no maintenance)

df2 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

#df2 = df2[~df2['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df2 = df2.dropna().copy() #eliminating low risk/unknown patients
df2 = df2.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df2 = df2.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df2 = df2.dropna(subset=['PFS_Status (1=censored,2=event)'])

#df2.to_csv("ASCT_PFS_HighRisk_NumOfMainten_Dataset.csv")


### PFS high risk patients, (Maintenance vs. no maintenance)

df22 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

#df22 = df22[~df22['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df22 = df22.dropna().copy() #eliminating low risk/unknown patients
df22 = df22.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df22 = df22.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df22 = df22.dropna(subset=['PFS_Status (1=censored,2=event)'])
df22['Number_of_Maintenance_Drugs'] = df22['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df22.to_csv("ASCT_PFS_HighRisk_MaintenVsNone_Dataset.csv")


### PFS high risk patients, (2 vs. 1 maintenance)
df222 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]
df222 = df222.dropna().copy() #eliminating low risk/unknown patients
df222 = df222.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df222 = df222.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df222 = df222.dropna(subset=['PFS_Status (1=censored,2=event)'])
df222 = df222[~df222['Number_of_Maintenance_Drugs'].isin([0])] #eliminating people with zero maintenance drugs 

#df222.to_csv("ASCT_PFS_HighRisk_1vs2Mainten_Dataset.csv")




### PFS, low risk patients, (Li + Pi vs. single vs. no maintenance)

df3 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

df3 = df3[~df3['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df3 = df3.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df3 = df3.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df3 = df3.dropna(subset=['PFS_Status (1=censored,2=event)'])

#df3.to_csv("ASCT_PFS_LowRisk_NumOfMainten_Dataset.csv")




### PFS, low risk patients, (Maintenance vs. no maintenance)

df33 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

df33 = df33[~df33['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df33 = df33.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df33 = df33.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df33 = df33.dropna(subset=['PFS_Status (1=censored,2=event)'])
df33['Number_of_Maintenance_Drugs'] = df33['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)


#df33.to_csv("ASCT_PFS_LowRisk_MaintenVsNone_Dataset.csv")





### OS, high risk patients, (Li + Pi vs. single vs. no maintenance)

df6 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

#df6 = df6[~df6['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df6 = df6.dropna().copy() #eliminating low risk/unknown patients
df6 = df6.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df6 = df6.dropna(subset=['Overall Survival (OS) (days)'])
df6 = df6.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df6.to_csv("ASCT_OS_HighRisk_NumOfMainten_Dataset.csv")



### OS, high risk patients, (Maintenance vs. no maintenance)

df66 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

#df66 = df66[~df66['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df66 = df66.dropna().copy() #eliminating low risk/unknown patients
df66 = df66.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df66 = df66.dropna(subset=['Overall Survival (OS) (days)'])
df66 = df66.dropna(subset=['OS_Status (1 = censored, 2 = event)'])
df66['Number_of_Maintenance_Drugs'] = df66['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)


#df66.to_csv("ASCT_OS_HighRisk_MaintenVsNone_Dataset.csv")





### OS, low risk patients, (Li + Pi vs. single vs. no maintenance)

df7 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

df7 = df7[~df7['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df7 = df7.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df7 = df7.dropna(subset=['Overall Survival (OS) (days)'])
df7 = df7.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df7.to_csv("ASCT_OS_LowRisk_NumOfMainten_Dataset.csv")




### OS, low risk patients, (Maintenance vs. no maintenance)

df77 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

df77 = df77[~df77['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df77 = df77.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df77 = df77.dropna(subset=['Overall Survival (OS) (days)'])
df77 = df77.dropna(subset=['OS_Status (1 = censored, 2 = event)'])
df77['Number_of_Maintenance_Drugs'] = df77['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)


#df77.to_csv("ASCT_OS_LowRisk_MaintenVsNone_Dataset.csv")





### OS, high risk, Li maintenance dose 
df8 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony', 'lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)']]

#df8 = df8[~df8['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df8 = df8.dropna().copy() #eliminating low risk/unknown patients
df8 = df8.dropna(subset=['lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)']) # Eliminating Nan values 
df8 = df8[~df8['lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)'].isin([1])] # Eliminating 'discontinued' rows
df8 = df8.dropna(subset=['Overall Survival (OS) (days)'])
df8 = df8.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df8.to_csv("ASCT_OS_HighRisk_LiMaintenDose_Dataset.csv")




### OS, low risk, Li maintenance dose 
df9 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony', 'lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)']]

df9 = df9[~df9['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df9 = df9.dropna(subset=['lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)']) # Eliminating Nan values 
df9 = df9[~df9['lenalidomide maintenance dose ( 1= discontinued, 2 = reduced, 3 = same)'].isin([1])] # Eliminating 'discontinued' rows
df9 = df9.dropna(subset=['Overall Survival (OS) (days)'])
df9 = df9.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df9.to_csv("ASCT_OS_LowRisk_LiMaintenDose_Dataset.csv")






### PFS2, high risk patients, (Li + Pi vs. single vs. no maintenance)

df10 = df[['STUDY_PATIENT_ID','Progression Free Survival 2 (PFS2) (days)','PFS2_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

#df10 = df10[~df10['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df10 = df10.dropna().copy() #eliminating low risk/unknown patients
df10 = df10.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df10 = df10.dropna(subset=['Progression Free Survival 2 (PFS2) (days)'])
df10 = df10.dropna(subset=['PFS2_Status (1 = censored, 2 = event)'])

#df10.to_csv("ASCT_PFS2_HighRisk_NumOfMainten_Dataset.csv")



### PFS2, high risk patients, (Maintenance vs. no maintenance)

df1010 = df[['STUDY_PATIENT_ID','Progression Free Survival 2 (PFS2) (days)','PFS2_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

#df1010 = df1010[~df1010['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df1010 = df1010.dropna().copy() #eliminating low risk/unknown patients
df1010 = df1010.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df1010 = df1010.dropna(subset=['Progression Free Survival 2 (PFS2) (days)'])
df1010 = df1010.dropna(subset=['PFS2_Status (1 = censored, 2 = event)'])
df1010['Number_of_Maintenance_Drugs'] = df1010['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df1010.to_csv("ASCT_PFS2_HighRisk_MaintenVsNone_Dataset.csv")


### PFS2, high risk patients, (1 vs. 2 maintenace)
df101010 = df[['STUDY_PATIENT_ID','Progression Free Survival 2 (PFS2) (days)','PFS2_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]
df101010 = df101010.dropna().copy() #eliminating low risk/unknown patients
df101010 = df101010.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df101010 = df101010.dropna(subset=['Progression Free Survival 2 (PFS2) (days)'])
df101010 = df101010.dropna(subset=['PFS2_Status (1 = censored, 2 = event)'])
df101010 = df101010[~df101010['Number_of_Maintenance_Drugs'].isin([0])] #eliminating people with zero maintenance drugs 

#df101010.to_csv("ASCT_PFS2_HighRisk_1vs2Mainten_Dataset.csv")





### PFS2, low risk patients, (Li + Pi vs. single vs. no maintenance)

df11 = df[['STUDY_PATIENT_ID','Progression Free Survival 2 (PFS2) (days)','PFS2_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

df11 = df11[~df11['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df11 = df11.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df11 = df11.dropna(subset=['Progression Free Survival 2 (PFS2) (days)'])
df11 = df11.dropna(subset=['PFS2_Status (1 = censored, 2 = event)'])

#df11.to_csv("ASCT_PFS2_LowRisk_NumOfMainten_Dataset.csv")





### PFS2, low risk patients, (Maintenance vs. no maintenance)

df1111 = df[['STUDY_PATIENT_ID','Progression Free Survival 2 (PFS2) (days)','PFS2_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','Number_of_Maintenance_Drugs']]

df1111 = df1111[~df1111['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df1111 = df1111.dropna(subset=['Number_of_Maintenance_Drugs']) #Eliminating Nan values in 'Number of maintenace drugs'
df1111 = df1111.dropna(subset=['Progression Free Survival 2 (PFS2) (days)'])
df1111 = df1111.dropna(subset=['PFS2_Status (1 = censored, 2 = event)'])
df1111['Number_of_Maintenance_Drugs'] = df1111['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df1111.to_csv("ASCT_PFS2_LowRisk_MaintenVsNone_Dataset.csv")








### PFS, high risk, (Single vs. tandem)
df12 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','TANDEM']]

#df12 = df12[~df12['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df12 = df12.dropna().copy() #eliminating low risk/unknown patients
df12 = df12.dropna(subset=['TANDEM']) #Eliminating Nan values in 'TANDEM'
df12 = df12.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df12 = df12.dropna(subset=['PFS_Status (1=censored,2=event)'])

#df12.to_csv('ASCT_PFS_HighRisk_SingVsTan.csv')




### PFS, low risk, (Single vs. tandem)
df13 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','TANDEM']]

df13 = df13[~df13['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df13 = df13.dropna(subset=['TANDEM']) #Eliminating Nan values in 'TANDEM'
df13 = df13.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df13 = df13.dropna(subset=['PFS_Status (1=censored,2=event)'])

#df13.to_csv('ASCT_PFS_LowRisk_SingVsTan.csv')





### OS, high risk, (Single vs. tandem)
df14 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','TANDEM']]

#df14 = df14[~df14['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df14 = df14.dropna().copy() #eliminating low risk/unknown patients
df14 = df14.dropna(subset=['TANDEM']) #Eliminating Nan values in 'TANDEM'
df14 = df14.dropna(subset=['Overall Survival (OS) (days)'])
df14 = df14.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df14.to_csv('ASCT_OS_HighRisk_SingVsTan.csv')




### OS, low risk, (Single vs. tandem)
df15 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','TANDEM']]

df15 = df15[~df15['At_Risk (Y/N) for Anthony'].isin(['Y'])] #eliminating high risk patients
df15 = df15.dropna(subset=['TANDEM']) #Eliminating Nan values in 'TANDEM'
df15 = df15.dropna(subset=['Overall Survival (OS) (days)'])
df15 = df15.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df15.to_csv('ASCT_OS_LowRisk_SingVsTan.csv')



### OS, (high vs. low risk) 

df16 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)', 'OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony']]
df16 = df16.dropna(subset=['Overall Survival (OS) (days)'])
df16 = df16.dropna(subset=['OS_Status (1 = censored, 2 = event)'])

#df16.to_csv("ASCT_OS_Risk_Dataset.csv")



### PFS, high vs. low risk

df17 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony']]
df17 = df17.dropna(subset=['Progression Free Surivival (PFS) (days)'])
df17 = df17.dropna(subset=['PFS_Status (1=censored,2=event)'])

#df17.to_csv("ASCT_PFS_Risk_Dataset.csv")




### PFS2, high vs. low risk 

df18 = df[['STUDY_PATIENT_ID','Progression Free Survival 2 (PFS2) (days)','PFS2_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony']]
df18 = df18.dropna(subset=['Progression Free Survival 2 (PFS2) (days)'])
df18 = df18.dropna(subset=['PFS2_Status (1 = censored, 2 = event)'])

#df18.to_csv("ASCT_PFS2_Risk_Dataset.csv")


##### Creating the figures Shona asked for 

### PFS, high risk, single transplant, (Maint. vs. not)

df19 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
#df19 = df19[~df19['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df19 = df19.dropna().copy() #eliminating low risk/unknown patients
df19 = df19[~df19['TANDEM'].isin(['YES'])] #eliminating Tandem patients (keeping only single transplants)
df19['Number_of_Maintenance_Drugs'] = df19['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df19.to_csv("ASCT_PFS_HighRisk_SingleTrans_MaintenVsNone_Dataset.csv")



### PFS, high risk, tandem transplant, (Maint. vs. not) 
df20 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
#df20 = df20[~df20['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df20 = df20.dropna().copy() #eliminating low risk/unknown patients
df20 = df20[~df20['TANDEM'].isin(['NO'])] #eliminating single transplant patients
df20['Number_of_Maintenance_Drugs'] = df20['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df20.to_csv("ASCT_PFS_HighRisk_TandemTrans_MaintenVsNone_Dataset.csv")



### OS, high risk, single transplant, (Maint. vs. not)

df21 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
#df21 = df21[~df21['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df21 = df21.dropna().copy() #eliminating low risk/unknown patients
df21 = df21[~df21['TANDEM'].isin(['YES'])] #eliminating Tandem patients (keeping only single transplants)
df21['Number_of_Maintenance_Drugs'] = df21['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df21.to_csv("ASCT_OS_HighRisk_SingleTrans_MaintenVsNone_Dataset.csv")



### OS, high risk, tandem transplant, (Maint. vs. not) 

df23 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
#df23 = df23[~df23['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df23 = df23.dropna().copy() #eliminating low risk/unknown patients
df23 = df23[~df23['TANDEM'].isin(['NO'])] #eliminating single transplant patients
df23['Number_of_Maintenance_Drugs'] = df23['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df23.to_csv("ASCT_OS_HighRisk_TandemTrans_MaintenVsNone_Dataset.csv")





### PFS, high risk, no maintenance, Single vs. Tandem 
df60 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
#df60 = df60[~df60['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df60 = df60.dropna().copy() #eliminating low risk/unknown patients
df60 = df60[~df60['Number_of_Maintenance_Drugs'].isin([1])] #eliminating people with 1 maintenace drug
df60 = df60[~df60['Number_of_Maintenance_Drugs'].isin([2])] #eliminating people with 2 maintenace drugs

#df60.to_csv("ASCT_PFS_HighRisk_NoMainten_SingVsTan_Dataset.csv")


### PFS, high risk, maintenance, single vs tandem 
df61 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
#df61 = df61[~df61['At_Risk (Y/N) for Anthony'].isin(['nan'])] #eliminating low risk/unknown patients
df61 = df61.dropna().copy() #eliminating low risk/unknown patients
df61 = df61[~df61['Number_of_Maintenance_Drugs'].isin([0])] #eliminating people with 0 maintenace drug

#df61.to_csv("ASCT_PFS_HighRisk_Mainten_SingVsTan_Dataset.csv")


### OS, high risk, no maintenance, Single vs. Tandem 

df62 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
df62 = df62.dropna().copy() #eliminating low risk/unknown patients
df62 = df62[~df62['Number_of_Maintenance_Drugs'].isin([1])] #eliminating people with 1 maintenace drug
df62 = df62[~df62['Number_of_Maintenance_Drugs'].isin([2])] #eliminating people with 2 maintenace drugs

#df62.to_csv("ASCT_OS_HighRisk_NoMainten_SingVsTan_Dataset.csv")

### OS, high risk, maintenance, single vs tandem 

df63 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','At_Risk (Y/N) for Anthony','TANDEM','Number_of_Maintenance_Drugs']]
df63 = df63.dropna().copy() #eliminating low risk/unknown patients
df63 = df63[~df63['Number_of_Maintenance_Drugs'].isin([0])] #eliminating people with 0 maintenace drug

#df63.to_csv("ASCT_OS_HighRisk_Mainten_SingVsTan_Dataset.csv")



##### Creating the figure Mina asked for 

### PFS, all risk, (Mainten vs None)
df64 = df[['STUDY_PATIENT_ID','Progression Free Surivival (PFS) (days)','PFS_Status (1=censored,2=event)','Number_of_Maintenance_Drugs']]
df64['Number_of_Maintenance_Drugs'] = df64['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df64.to_csv("ASCT_PFS_MaintenVSNone_Dataset.csv")



### OS, all risk (Mainten vs None)
df65 = df[['STUDY_PATIENT_ID','Overall Survival (OS) (days)','OS_Status (1 = censored, 2 = event)','Number_of_Maintenance_Drugs']]
df65['Number_of_Maintenance_Drugs'] = df65['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)

#df65.to_csv("ASCT_OS_MaintenVSNone_Dataset.csv")









<ipython-input-5-4a851235418d>:419: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df64['Number_of_Maintenance_Drugs'] = df64['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintenace = 1, and no maintenance = 0)
<ipython-input-5-4a851235418d>:427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df65['Number_of_Maintenance_Drugs'] = df65['Number_of_Maintenance_Drugs'].replace([2],1) # replacing number of maintenance= 2 to =1 (i.e. category of maintena

Creating the data file that will be used for the cox regression analysis analysis 

In [6]:
### Eliminating unimportant columns
df_multi = df.drop(columns=['STUDY_PATIENT_ID', 'Neutrophils','Calcium','REGIMEN_NAME_maintenance','Response post induction', 'ASCT response  1-3 months', 'ASCT response 12 months', 'Response post 2nd induction', 'ASCT response  1-3 months (second transplant)', 'ASCT response 12 months (second transplant)', 'Tandem 1 response 1-3 months', 'Tandem 2 response 1-3 months', 'Tandem 2 response 12 months', 'Maint. Response 12 months', 'Maint. Response 12 months (2nd transplant)', 'Maint. Type (after Tandem)', 'Paatient Status', 'Comments'])

#Dropping rows that have some errors.
df_multi = df_multi.drop(labels=172, axis=0)
df_multi = df_multi.drop(labels=173, axis=0)
df_multi = df_multi.drop(labels=174, axis=0)

#Convert categorical data into numerical representation 
number = LabelEncoder()
df_multi['GENDER'] = number.fit_transform(df_multi['GENDER'].astype('str'))
df_multi['At_Risk (Y/N) for Anthony'] = df_multi['At_Risk (Y/N) for Anthony'].replace(['Y'],1)
df_multi['At_Risk (Y/N) for Anthony'].fillna(0, inplace=True)
df_multi['TANDEM'] = number.fit_transform(df_multi['TANDEM'].astype('str'))

df_multi['Hemoglobin'] = df_multi['Hemoglobin'].astype(str)
df_multi['Creatinine'] = df_multi['Creatinine'].astype(str)

#Remove the units from the hemoglobin and creatinine concentrations
unit = 'g/L'
df_multi['Hemoglobin'] = [sub.replace(unit,"").strip() for sub in df_multi['Hemoglobin']]
unit2 = 'umol/L'
df_multi['Creatinine'] = [sub.replace(unit2,"").strip() for sub in df_multi['Creatinine']]

#Normally would save the file
#df_multi.to_csv('ASCT_Multivar.csv')


